### Capstone project - The battle of neighborhoods

### Import libraries

In [75]:
import numpy as np

import pandas as pd
from pandas.io.json import json_normalize 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests
import folium 

### Connect to foursquare API

In [76]:
CLIENT_ID = 'AXX3WHYZ5VLW12O30F01F33RB0HMWVLE3ABC2TYOWATKLKIT' # my Foursquare ID
CLIENT_SECRET = 'B1WZ2FNPCJMWTCXNX5MYUYUQSMNFH3XWJOX5F5ZIH4QGZ0DN' # my Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

### Get the data from foursquare

In [77]:
# type your answer here
LIMIT = 500 # Maximum is 100
cities = ["Amsterdam", 'Rotterdam', 'Breda', 'Utrecht', 'Den Haag']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d1d3941735") # VEGGIE DINER
    results[city] = requests.get(url).json()

#### Get the closest venues

In [78]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

<ipython-input-78-7134362d7262>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  venues = json_normalize(results[city]['response']['groups'][0]['items'])


In [91]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of vegetarion places in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of vegetarion places in Amsterdam =  89
Showing Top 100
Total number of vegetarion places in Rotterdam =  23
Showing Top 100
Total number of vegetarion places in Breda =  10
Showing Top 100
Total number of vegetarion places in Utrecht =  10
Showing Top 100
Total number of vegetarion places in Den Haag =  11
Showing Top 100


In [80]:
maps[cities[0]]

In [81]:
maps[cities[1]]

In [82]:
maps[cities[2]]

In [92]:
maps[cities[3]]

In [84]:
maps[cities[4]]

### let's analyze the distance

In [85]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

Amsterdam
Mean Distance from Mean coordinates
0.023424863185122456
Rotterdam
Mean Distance from Mean coordinates
0.01766483523518664
Breda
Mean Distance from Mean coordinates
0.20747307730301187
Utrecht
Mean Distance from Mean coordinates
0.020034368946965954
Den Haag
Mean Distance from Mean coordinates
0.016539508830651903


In [86]:
maps[cities[0]]

In [87]:
maps[cities[1]]

In [88]:
maps[cities[2]]

In [89]:
maps[cities[3]]

In [90]:
maps[cities[4]]